This script processes plan, payment, history, and game-list data to construct subscription periods, enrich them with game/platform categories, and export subscription-level features for analysis.

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

### Functions

In [ ]:
def make_adj_end_date(plan: pd.DataFrame) -> pd.DataFrame:
    """Calculates the adjusted end date for each subscription plan based on official end date."""

    plan_tmp = plan.copy()
    plan_tmp = plan_tmp.sort_values(by=['id', 'start_date', 'end_date', 'cancel_date'])
    plan_tmp['adj_end_date'] = np.where(plan_tmp['cancel_date'] < plan_tmp['end_date'], plan_tmp['cancel_date'], plan_tmp['end_date'])
    plan_tmp.loc[(plan_tmp['cancel_date'].notna() & plan_tmp['adj_end_date'].isna()), 'adj_end_date'] = plan_tmp['cancel_date']

    na_counts = plan_tmp['adj_end_date'].isna().groupby(plan_tmp['id']).transform('sum')
    has_at_most_one_na = na_counts <= 1
    multi_end_date = plan_tmp.loc[~has_at_most_one_na, 'id'].drop_duplicates()
    plan_tmp = plan_tmp[~plan_tmp['id'].isin(multi_end_date)]

    return plan_tmp

def make_subscription(plan: pd.DataFrame) -> pd.DataFrame:
    """Groups individual plan records into continuous subscription periods based on time gaps."""

    plan_tmp = plan.copy()
    plan_tmp = plan_tmp.sort_values(by=['id', 'adj_end_date'])
    plan_tmp['next_start_day'] = plan_tmp.groupby('id')['start_date'].shift(-1)
    plan_tmp['gap_next_start_day'] = np.where(plan_tmp['adj_end_date'].notna(), plan_tmp['next_start_day'] - plan_tmp['adj_end_date'], plan_tmp['next_start_day'] - plan_tmp['start_date'])
    plan_tmp['one_month_gap'] = plan_tmp['gap_next_start_day'].dt.days > 30
    plan_tmp['last_row'] = plan_tmp.groupby("id")['id'].transform(lambda x: x.shift(-1).isna())
    plan_tmp = plan_tmp.sort_values(by=['id', 'start_date', 'adj_end_date'])

    subscription_list = []
    for id in tqdm(plan_tmp['id'].unique()):
        period_start = None
        period_end = None

        # Iterate through each unique ID to group plans into continuous subscriptions.
        for _, row in plan_tmp[plan_tmp['id'] == id].sort_values(by="start_date").iterrows():
            if period_start is None:
                period_start = row['start_date']

            if row['one_month_gap'] == True or row['last_row'] == True:
                period_end = row['adj_end_date']
                subscription_list.append({'id': id, 'sub_start': period_start, 'sub_end': period_end})
                period_start = None        
            
    subscription = pd.DataFrame(subscription_list)
    subscription['loss'] = np.where(subscription['sub_end'].notna(),1, 0)
    subscription['sub_end'] = subscription['sub_end'].fillna(pd.Timestamp('2024-01-22'))
    subscription['sub_days'] = (subscription['sub_end'] - subscription['sub_start']).dt.days
    subscription = subscription[subscription['sub_days'] > 7]
    subscription = subscription.reset_index(drop=True)

    return subscription


def plan_type(subscription: pd.DataFrame, plan: pd.DataFrame, plan_recon: pd.DataFrame) -> pd.DataFrame:
    """Determines the most frequent plan type for each consolidated subscription period."""

    new_plan = pd.merge(plan, plan_recon, left_on='plan', right_on='方案名稱', how='left')
    new_plan.drop_duplicates(inplace=True, keep='first')
    new_plan = new_plan.drop(columns=['申請通路', '方案名稱', "cancel_date", "remark", "channel", "promo", "end_date"])
    new_plan.rename(columns={'方案名稱(彙整)': 'plan_name'}, inplace=True)

    new_plan['plan_name'] = new_plan['plan_name'].astype(str)
    new_plan['plan_type'] = new_plan['plan_name'].apply(lambda x: x.split(' ')[0][-2:])
    new_plan['plan_inval'] = new_plan.apply(lambda row: pd.Interval(left=row['start_date'], right=row['adj_end_date'], closed='both'), axis=1)


    for i in range(len(subscription)):
        cur_id = subscription.loc[i, 'id']
        cur_start = pd.Timestamp(subscription.loc[i, 'sub_start_new'])
        cur_end = pd.Timestamp(subscription.loc[i, 'sub_end'])

        cur_id_plan_df = new_plan[new_plan["id"] == cur_id]
        mask = cur_id_plan_df.apply(lambda row: (cur_start in row['plan_inval']) | (cur_end in row['plan_inval']), axis=1)
        freq_plan_type = cur_id_plan_df[mask]["plan_type"].value_counts().idxmax()
        subscription.loc[i, "plan_type"] = freq_plan_type
    
    return subscription


def process_data_connect_platform_game_category(subscription, history, gamelist):
    """Calculates subscription features and aggregates user's most frequent behavior (connect, game, platform) and vacation features."""

    # Calculate 'sub_start_new': If subscription is longer than 12 weeks (~90 days),
    # the new start date is 12 weeks before 'sub_end', otherwise it's 'sub_start'.
    subscription['sub_start_new'] = subscription.apply(lambda row: row['sub_start'] if row['sub_end'] - pd.DateOffset(days=7*12) < row['sub_start'] else row['sub_end'] - pd.DateOffset(days=7*12), axis=1)
    subscription = subscription[['id','sub_start', 'sub_end', 'loss', 'sub_start_new', 'sub_days']]
    subscription.loc[:, 'unique_id'] = subscription.index

    history["play_date"] = pd.to_datetime(history["play_date"])

    # Create a dictionary to store the most frequent connect, platform, and game usage for each subscription period.
    game_dict = {}

    for id in tqdm(subscription['unique_id']):
        week_game = {}
        history_data = history[(history['id'] == subscription.iloc[id]["id"]) & (history['play_date'] < subscription.iloc[id]['sub_end']) & (history['play_date'] >= subscription.iloc[id]['sub_start_new'])]
        most_common_connect = history_data['connect'].mode().iloc[0] if not history_data['connect'].empty and not history_data['connect'].mode().empty else None
        most_common_platform = history_data['platform'].mode().iloc[0] if not history_data['platform'].empty and not history_data['platform'].mode().empty else None
        most_common_game = history_data['game'].mode().iloc[0] if not history_data['game'].empty and not history_data['game'].mode().empty else None
        
        week_game["connect"] = most_common_connect
        week_game["platform"] = most_common_platform
        week_game["game"] = most_common_game

        # --- Vacation/Holiday Feature Engineering ---
        sub_start  = subscription.iloc[id]["sub_start"].month
        sub_end = subscription.iloc[id]["sub_end"].month

        # Check if the subscription started before typical vacation months (June, December).
        if sub_start == 6 or sub_start == 12:
            week_game["sub_before_vacation"] = True
        else: 
            week_game["sub_before_vacation"] = False

        # Check if the subscription started during typical vacation months (Jan, July, August).
        if sub_start == 1 or sub_start == 7 or sub_start == 8:
            week_game["sub_during_vacation"] = True
        else:
            week_game["sub_during_vacation"] = False

        # Check if the subscription ended after typical vacation months (Feb, Sept).
        if sub_end == 2 or sub_end == 9:
            week_game["unsub_after_holiday"] = True
        else:
            week_game["unsub_after_holiday"] = False

        # Check if the subscription ended during typical vacation months (Jan, July, August).
        if sub_end == 1 or sub_end == 7 or sub_end == 8:
            week_game["unsub_during_holiday"] = True
        else:
            week_game["unsub_during_holiday"] = False

        # Check if the subscription period covers any vacation month (Jan, July, August).
        if sub_end < sub_start:
            months_range = list(range(sub_start, 13)) + list(range(1, sub_end + 1))
        else:
            months_range = list(range(sub_start, sub_end + 1))
        if 7 in months_range or 8 in months_range or 1 in months_range:
            week_game["cover_vacation"] = True
        else:
            week_game["cover_vacation"] = False

        game_dict[id] = week_game

    game_df = pd.DataFrame.from_dict(game_dict, orient='index')
    unique_gamelist = gamelist.drop_duplicates(subset='game')
    unique_gamelist = unique_gamelist[['game', 'casual', 'free', 'adventure', 'action', 'multiplayer_tactical_competition', 'massive_multiplayer_online', 'platformer','fighting','simulation', 'indie', 'racing', 'first_person_shooter', 'strategy', 'arcade', 'role_playing', 'puzzle', 'sports', 'family']]
    merged_df = pd.merge(game_df, unique_gamelist, on='game', how='left')
    output = pd.concat([subscription, merged_df], axis=1)

    return output

In [12]:
plan = pd.read_csv('data/plan.csv', parse_dates=['start_date', 'end_date', 'cancel_date'])
history = pd.read_csv('data/history.csv')
payment = pd.read_csv('data/payment.csv')
info = pd.read_csv('data/info.csv')
plan_recon = pd.read_csv('data/方案對照表_cleaned.csv')
game_list = pd.read_csv('data/gamelist.csv')

In [13]:
new_plan = make_adj_end_date(plan)

In [ ]:
subscription = make_subscription(new_plan)
new_plan['adj_end_date'] = new_plan['adj_end_date'].fillna(pd.Timestamp('2024-01-22'))

100%|██████████| 36551/36551 [00:58<00:00, 621.95it/s]


In [ ]:
subscription = process_data_connect_platform_game_category(subscription, history, game_list)
subscription = plan_type(subscription, new_plan, plan_recon)
subscription.to_csv('data/subscription.csv', index=False)

/var/folders/vj/9zg_mvkx2rj9gn5htgdt27080000gn/T/ipykernel_77824/3527779356.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subscription.loc[:, 'unique_id'] = subscription.index
100%|██████████| 43474/43474 [01:12<00:00, 599.25it/s]
